# Accessing Free Hourly Weather Data
NOAA stores Free Hourly Weather Data at: ftp://ftp.ncdc.noaa.gov/pub/data/noaa/

It is my understanding that this is the data underlying most of the websites that charge for historical weather data at the hourly level (at least when needed in bulk).

The most detailed data is in a very cumbersome format, but a subset of easy to parse data can be found at: ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-lite/

To make this topic approachable to a large audience, I've added some notes on how one could access this data simply using Excel at the end of this notebook.

In [160]:
# Column Names were determined from ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-lite/isd-lite-format.pdf
# That pdf describes what data is contained in the subset of data that I'll focus on.
isd_fwf_cols = ['year', 'month', 'day', 'hour', 'air_temp_c', 'dew_pt_temp_c',
                 'sea_lvl_press_hectoPa', 'wnd_dir_360', 'wnd_spd_mtrpersec',
                 'sky_condition', 'precip_hrly', 'precip_6hr_accum']

In [161]:
# Importing the python libraries that I use.
import pandas as pd
import numpy as np

In [162]:
# Importing the table defining the available data. 
# There is a row for each station and it includes the begin and end date of available data.
isd_stations_data = pd.read_csv('ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-history.csv')

In [163]:
# I want data for DC, so I've chosen to search for the local airport. Reagan National Airport (DCA).
# Note that all of the Station Names are uppercase.
DCA_search = isd_stations_data.loc[(isd_stations_data['STATION NAME'].isna() == False) 
                                   & (isd_stations_data['STATION NAME'].str.contains('REAGAN'))]

In [164]:
# Slicing out the BEGIN and END years to create the range of years for which I'll download data.
start_year = str(list(DCA_search.BEGIN)[0])[0:4]
end_year = str(list(DCA_search.END)[0])[0:4]
year_range = range(int(start_year), int(end_year)+1)

In [165]:
# Creating the station ID by which the ftp site is organized.
# Note that it is the concatenation of two columns separated by a hyphen.
station_id = str(list(DCA_search.USAF)[0])+'-'+str(list(DCA_search.WBAN)[0])

In [144]:
# Function to loop through a given station ID for a given range of years.
def download_isd_lite(station_id, year_range):
    isd_df = pd.DataFrame()
    for year in year_range:
        # There can be gaps of missing years in the data, so try and except were required. 
        # The gaps that I've seen are only from decades ago.
        try:
            new_isd_df = pd.read_fwf('ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-lite/'
                                     +str(year)+'/'
                                     +station_id+'-'
                                     +str(year)
                                     +'.gz',
                                     header=None)
            isd_df = pd.concat([isd_df, new_isd_df])
        except:
            continue
    
    # Resetting the index of the concatenated DataFrame
    isd_df.reset_index(inplace=True, drop=True)
    
    # Setting the column names that I've derived from the format guide
    isd_df.columns = isd_fwf_cols
   
    # NOAA populates missing values with -9999, but I've chosen to replace them with NaN's.
    isd_df.replace({-9999: np.nan}, inplace=True)
    
    # Some of the columns are scaled by a factor of 10 to eliminate decimal points,
    # which would complicate the fixed width format that NOAA has chosen to utilize
    scaled_columns = ['air_temp_c', 'dew_pt_temp_c', 'sea_lvl_press_hectoPa', 
                  'wnd_spd_mtrpersec', 'precip_hrly', 'precip_6hr_accum']
    scaling_factor = 10
    # Resolving the scaling factor
    isd_df[scaled_columns] = isd_df[scaled_columns] / 10
    
    # Creating a date_time column from the various time-based columns NOAA provides.
    # The first step is creating a properly formatted string that pandas can parse, and then parse them.
    isd_df['date_time'] = isd_df.day.astype('int').astype('str').str.zfill(2)+'/'\
                         +isd_df.month.astype('int').astype('str').str.zfill(2)+'/'\
                         +isd_df.year.astype('int').astype('str')+'/'\
                         +isd_df.hour.astype('int').astype('str').str.zfill(2)
    isd_df['date_time'] = pd.to_datetime(isd_df['date_time'], format='%d/%m/%Y/%H')
    
    return isd_df

In [145]:
# Running the function for DCA for all years. This data frame can then be manipulated or exported.
isd_df = download_isd_lite(station_id, year_range)

# Excel Users
You could create the ftp URLs in Excel and then manually click all the links you create. If you had a column with the station_id and a column with the year for each file you want then you could construct the URLs as follows:

=HYPERLINK("ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-lite/"&reference_year_cell&"/"&reference_station_id_cell&"-"&reference_year_cell&".gz")

You can then unzip each file, and open it in Excel. Using Excel's "Text to Columns" feature with the "Original data type" option set to "Fixed width", Excel will correctly separate the data in to columns. You can then manually add the column headers and save the data as an Excel file as desired. Then you could manually aggregate the data for multiple years and/or stations as needed.

# Mapping the 1000 Active US Airport Weather Stations with the Longest Recorded History

The map created below shows active (defined as having reported weather data in the last 2 days) weather stations at US airports. To ensure the map could be rendered, only 1000 weather stations were selected and those 1000 were selected based on having the oldest beginning date of data availability.

In [150]:
import folium

In [151]:
# Removing weather stations without valid coordinates
stations_with_coordinates = isd_stations_data.loc[((isd_stations_data.LAT != 0) 
                                                  & (isd_stations_data.LON != 0))
                                                  & (isd_stations_data.LAT.isna() == False)
                                                  & (isd_stations_data.LON.isna() == False)].copy()

In [152]:
# creating a list of the latitude and longitude coordinates, which will be fed into the mapping function
stations_with_coordinates['COORDINATES'] = list(zip(round(stations_with_coordinates.LAT,3), 
                                                    round(stations_with_coordinates.LON,3)))

In [153]:
# Trying to reduce the volume so a reasonable map can be rendered, and only active sites are included
most_recent_day = stations_with_coordinates['END'].max()
us_airports_with_coordinates = stations_with_coordinates.loc[(stations_with_coordinates.CTRY == 'US')
                                                             & (stations_with_coordinates.ICAO.isna() == False)
                                                             & (stations_with_coordinates.END > (most_recent_day-2))].copy()

In [154]:
# creating a string with the plain language Station Name and the Station ID used in the ftp site, 
# which will be presented on the map
us_airports_with_coordinates['IDENTIFIER'] = (us_airports_with_coordinates['STATION NAME']+'; Station ID: '
                                              +us_airports_with_coordinates['USAF'].astype('str')+'-'
                                              +us_airports_with_coordinates['WBAN'].astype('str'))

In [155]:
# sorting data so that when I trim the number of entries using .head(), I preserve those with the longest history of data
us_airports_with_coordinates.sort_values(by=['BEGIN'], ascending=True, inplace=True)

In [156]:
# Creating a map of the selected airport weather stations with annotations
base_map = folium.Map([52, -112], zoom_start=3)
# Limiting the map to 1000 locations
for index, coord in enumerate(us_airports_with_coordinates['COORDINATES'].head(1000)):
    marker = folium.Marker([coord[0],coord[1]], popup=us_airports_with_coordinates.iloc[index]['IDENTIFIER']).add_to(base_map)

In [159]:
# Here's the interactive map that you can use to find the Station ID for a given US location.
# I've commented it out because because it causes rendering issues in Medium
# base_map.save('1000_US_Airports_Weather_Stations_Map.html')
# base_map